In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score  
import pandas_profiling

from sklearn.tree import DecisionTreeClassifier, plot_tree

In [45]:
plt.rcParams['figure.figsize'] = (12, 6)

#This is global so if I don't specify figure size on each graph, this size is automatically assigned
#But also I can specify on each graph locally like this. plt.figure(figsize=(12,8)) and it will be overwritten

In [46]:
df = pd.read_csv('train.csv', sep = ',')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [47]:
#missing value check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Use descision tree model

In [48]:
# feature engineering for numericals
numeric_features = ['Age', 'Fare']

In [49]:
numeric_transformer = make_pipeline(
    SimpleImputer(strategy="mean"), 
    StandardScaler()
    )

#Need to deep dive

In [50]:
# split to X, and y
X = df.iloc[:,2:]
y = df['Survived']

#Why iloc[:,1:]?

In [53]:
# feature engineering for categorical
categorical_features = ['Embarked', 'Sex', 'Pclass']

In [54]:
# handle unknown means it will ignore nan's if it finds them
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

#If train data has all the possible categories, even if test data doesn' have all it's no problem. Just a column which has only 0 is created.
#If train data doesn't have all the possible categories but the test data has the missing one, the data is ignored and the column is not created as train data isn't

#pd.get_dummies only creates columns which are existing cateogories in the test data(even if train data has other categories) 
#Not sure what would happen if train data has more columns than test data after pd.get_dummies and vice versa

In [55]:
# pre-process transformations
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        #('name', FunctionTransformer(name_length), ['Name'])
    ],
    remainder='drop') 

#or passthrough. default 'drop'
#Why FunctionTransformer() has name_lengh def. How it works?
#Is it possible to specify columns which I wanna drop?

In [56]:
# create the model pipeline
pipeline = make_pipeline(preprocessor, DecisionTreeClassifier(max_depth=2, criterion='entropy'))

In [57]:
# split data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state= 42)

#random_state=42 means when I debug code, every time I get exactly same split between train data and test data
#Could be any number but same number is same split everytime every person

In [58]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Embarked', 'Sex',
                                                   'Pclass'])])),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(criterion='entropy', max_depth=2))])

## Make prediction

In [59]:
ypred = pipeline.predict(X_train)

In [60]:
ypred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [61]:
pipeline.score(X_train, y_train)

0.8108614232209738

In [62]:
pipeline.score(X_test, y_test)

0.773109243697479

In [63]:
ypred = pipeline.predict(X_test) 
accuracy_score(y_test,ypred) 
#same

0.773109243697479

## Plot the tree

In [64]:
t= plot_tree(pipeline.named_steps['decisiontreeclassifier'])
# plot_tree(pipeline.named_steps['DecisionTreeClassifier']) doesn't work because after fitting the pipline, steps=[] is created inside of pipeline
# and have to select the decision tree model only
# and decision tree model is converted to ('decisiontreeclassifier', DecisionTreeClassifier(criterion='entropy', max_depth=5) in this pipeline
# so have to specify as 'decisiontreeclassifier'


## Kaggle submission

In [65]:
test = pd.read_csv('test.csv')
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [68]:
#Create a DataFrame with the passengers ids and prediction regarding whether they survived or not
X_k = test.iloc[:,1:]
X_k
#features = ['Embarked', 'Sex', 'Pclass', 'Age', 'Fare']
predictions = pipeline.predict(X_k)

submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':predictions})

In [69]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [70]:
#Convert DataFrame to a csv file that can be uploaded
filename = 'Titanic Predictions Decision Tree.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: Titanic Predictions Decision Tree.csv
